In [1]:
import pandas as pd
import numpy as np
import yaml
from time import sleep
from pathlib import Path

from utils import get_season, nfl_weeks_pull, game_keys_pull, reg_season, post_season
from yahoo_query import league_season_data

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]

try:
    with open(PATH) as file:
        CREDS = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

# TODAY = np.datetime64("today", "D")

# TODAY = np.datetime64("2022-01-05")
# TODAY = np.datetime64("2022-01-02")
# TODAY = np.datetime64("2021-12-28")
# TODAY = np.datetime64("2021-12-21")
# TODAY = np.datetime64("2021-12-14")
# TODAY = np.datetime64("2021-12-07")
# TODAY = np.datetime64("2021-11-30")
# TODAY = np.datetime64("2021-11-23")
# TODAY = np.datetime64("2021-11-16")
# TODAY = np.datetime64("2021-11-09")
# TODAY = np.datetime64("2021-11-02")
# TODAY = np.datetime64("2021-10-26")
# TODAY = np.datetime64("2021-10-19")
# TODAY = np.datetime64("2021-10-12")
# TODAY = np.datetime64("2021-10-05")
# TODAY = np.datetime64("2021-09-28")
# TODAY = np.datetime64("2021-09-21")
# TODAY = np.datetime64("2021-09-14")
# TODAY = np.datetime64("2021-09-09")
TODAY = np.datetime64("2021-08-31")

YEAR = TODAY.astype("datetime64[Y]").astype(int) + 1970
NFL_WEEKS = nfl_weeks_pull()
MAX_WEEK = NFL_WEEKS["week"].max()
GAME_KEYS = game_keys_pull(first="no")
# SEASON = get_season()
SEASON = 2021

try:
    LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
except Exception as e:
    print(e)
GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]

try:
    NFL_WEEK = NFL_WEEKS["week"][
        (NFL_WEEKS["end"] >= TODAY) & (NFL_WEEKS["start"] <= TODAY)
    ].values[0]

except Exception as e:
    print(e)
    NFL_WEEK = np.nan


CONSUMER_KEY = CREDS["YFPY_CONSUMER_KEY"]
CONSUMER_SECRET = CREDS["YFPY_CONSUMER_SECRET"]

league = league_season_data(
    auth_dir=PATH.parent,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)
print(TODAY)
print(NFL_WEEK)
if TODAY == np.datetime64(f"{YEAR}-08-31"):
    game_keys = league.all_game_keys()
    nfl_weeks = league.all_nfl_weeks()
    meta = league.metadata(first_time="no")
    settings, roster, stat_cat = league.set_roster_pos_stat_cat(first_time="no")
    # players = league.players_list(first_time="no")
    teams = league.teams_and_standings(first_time="no")

if TODAY == NFL_WEEKS["end"][(NFL_WEEKS["week"] == MAX_WEEK) & (NFL_WEEKS["game_id"] == GAME_ID)].values[0]:
    teams = league.teams_and_standings(first_time="no")

if TODAY == NFL_WEEKS["start"][(NFL_WEEKS["game_id"] == GAME_ID) & (NFL_WEEKS["week"] == 1)].values[0]:
    draft = league.draft_results(first_time="no")

if 1 <= NFL_WEEK <= 17:
    week_roster = league.team_roster_by_week(first_time="no", nfl_week=NFL_WEEK)
    matchups = league.matchups_by_week_regseason(first_time="no", nfl_week=NFL_WEEK)
    team_points = league.team_points_by_week(first_time="no", nfl_week=NFL_WEEK)
    reg_season_board = reg_season(GAME_ID, NFL_WEEK)
    post_season_board = post_season(reg_season_board, GAME_ID, NFL_WEEK)

[2022-08-12 20:48:01,416 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fed4737b240; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7fed4737b240; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-08-12 20:48:01,419 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-08-12 20:48:01,420 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-08-12 20:48:01,422 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-08-12 20:48:01,422 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

Successfully pulled: SELECT * FROM dev.nfl_weeks
Successfully pulled: SELECT * FROM dev.game_keys
index 0 is out of bounds for axis 0 with size 0
2021-08-31
nan


[2022-08-12 20:48:02,520 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fed46320680; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7fed46320680; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-08-12 20:48:02,523 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-08-12 20:48:02,523 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-08-12 20:48:02,524 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-08-12 20:48:02,525 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

Upload successful: game_keys
Successfully pulled: SELECT game_id FROM dev.game_keys


[2022-08-12 20:48:05,375 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fed46320f40; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7fed46320f40; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-08-12 20:48:05,377 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-08-12 20:48:05,378 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-08-12 20:48:05,379 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-08-12 20:48:05,379 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

...

Successfully pulled: SELECT * FROM raw.league_settings WHERE (game_id <> '406') GROUP BY cant_cut_list, draft_pick_time, draft_time, draft_type, game_id, has_multiweek_championship, has_playoff_consolation_games, is_auction_draft, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, pickem_enabled, player_pool, playoff_start_week, post_draft_players, scoring_type, trade_end_date, trade_ratify_type, trade_reject_time, uses_faab, uses_fractional_points, uses_lock_eliminated_teams, uses_negative_points, uses_playoff, uses_playoff_reseeding, waiver_rule, waiver_time, waiver_type 
Upload successful: league_settings
Successfully pulled: SELECT * FROM raw.roster_positions WHERE (game_id <> '406') GROUP BY count, game_id, league_id, position, position_type 
Upload successful: roster_positions


[2022-08-12 20:48:07,238 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fed46321800; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=raw'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7fed46321800; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=raw'', closed: 0>
[2022-08-12 20:48:07,240 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-08-12 20:48:07,241 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-08-12 20:48:07,242 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-08-12 20:48:07,243 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

Successfully pulled: SELECT * FROM raw.stat_categories WHERE (game_id <> '406') GROUP BY display_name, enabled, game_id, league_id, name, position_type, sort_order, stat_id, stat_modifier, is_only_display_stat 
Upload successful: stat_categories


[2022-08-12 20:48:08,089 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fed463211c0; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=raw'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7fed463211c0; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=raw'', closed: 0>
[2022-08-12 20:48:08,091 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-08-12 20:48:08,092 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-08-12 20:48:08,093 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-08-12 20:48:08,094 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

Successfully pulled: SELECT * FROM raw.league_teams WHERE (game_id <> '406') GROUP BY clinched_playoffs, draft_grade, faab_balance, game_id, has_draft_grade, league_id, manager_id, name, nickname, number_of_moves, number_of_trades, "roster_adds.coverage_value", "roster_adds.value", team_id, team_key, "team_standings.outcome_totals.losses", "team_standings.outcome_totals.percentage", "team_standings.outcome_totals.ties", "team_standings.outcome_totals.wins", "team_standings.playoff_seed", "team_standings.points_against", "team_standings.points_for", "team_standings.rank", "team_standings.streak.type", "team_standings.streak.value", waiver_priority
Upload successful: league_teams


In [4]:
meta

,allow_add_to_dl_extra_pos,current_week,draft_status,edit_key,end_date,end_week,game_code,iris_group_chat_id,is_cash_league,is_finished,is_pro_league,league_id,league_key,league_type,league_update_timestamp,logo_url,name,num_teams,password,renew,renewed,scoring_type,season,short_invitation_url,start_date,start_week,url,weekly_deadline,game_id
0,0,17,postdraft,17,2022-01-03,17,nfl,None,0,1,0,103661,406.l.103661,private,1642058898,https://yahoofantasysports-res.cloudinary.com/...,Men of Madison,10,None,399_12514,414_77446,head,2021,https://football.fantasysports.yahoo.com/2021/...,2021-09-09,1,https://football.fantasysports.yahoo.com/2021/...,None,406


In [3]:
pd.set_option('display.max_columns', None)